In [5]:
# Variable
W_max = 19.5;
W_min = 10;
import xlrd
import xlsxwriter
import os


## excel parameter
# parameter schedule end Index
Schedule_endIndex = 289
# student schedule end Index
EndTime = 39;

# function for formating
def timeFormating(num):
    n = str(num)
    time = n.split(".")
    if time[1]=='5':
        time[1]='50' 
    min=str(int(time[1])*60/100).split(".")[0]
    if min =='0':
        min='00'
    if int(time[0]) <12:
        return time[0]+":"+min+"am"
    elif int(time[0])==12:
        return time[0]+":"+min+"pm"
    elif int(time[0])> 24:
        return str(int(time[0])-24)+":"+min+"am"
    elif int(time[0]) >12:
        return str(int(time[0])-12)+":"+min+"pm"
    
def dayFormating(num):
    if num==1:
        return "Mon"
    elif num ==2:
        return "Tue"
    elif num==3:
        return "Wed"
    elif num==4:
        return "Thu"
    elif num==5:
        return "Fri"
    elif num==6:
        return "Sat"
    elif num==7:
        return "Sun"

loc = ("schedule.xls")
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
x = list(range(1, Schedule_endIndex))
#initial data
startTime = []
processStartTime = []
endTime = []
shiftslength = []
days = []
shiftstations = []
station = []

# get shift data
for i in x:
    timeStarted = sheet.cell_value(i,1)
    startTime.append(timeStarted)
    if str(timeStarted)[-3:] ==".25" or str(timeStarted)[-3:] ==".75" :
        processStartTime.append(timeStarted-0.25)
    else :
        processStartTime.append(timeStarted)  
    endTime.append(sheet.cell_value(i,2))
    shiftslength.append(sheet.cell_value(i,3))
    days.append(sheet.cell_value(i,4))
    shiftstations.append(sheet.cell_value(i,5))
    if sheet.cell_value(i,5) not in station:
        station.append(sheet.cell_value(i,5))

shifts = list(range(0, Schedule_endIndex-1))
# get all students availability and prefer time
studentName = []
p = []
#allAvailability[i]:availability for student i
#allAvailability[i][j]:availability for student i on day j
#allAvailability[i][j][k]:availability for student i on day j at time k
allAvailability = []
dirs = os.listdir("studentAvailability")
for file in dirs:
    if (not file.startswith("~$")) and (not file.startswith(".")) and (not file.startswith("_")):
        studentName.append(file.split(".")[0])
        # get each student availability
        wbp=xlrd.open_workbook("studentAvailability/"+file)
        sheetperson = wbp.sheet_by_index(0)
        # get prefer working time
        p.append(sheetperson.cell_value(1,8))
        time_slot = list(range(1, EndTime))
        stuAvailability = []
        for j in list(range(1,8)):
            day_slot = []
            for i in time_slot:
                day_slot.append(sheetperson.cell_value(i,j))
            stuAvailability.append(day_slot)
        allAvailability.append(stuAvailability)
# get whether shift is available for each student
a=[]
for i in range(len(studentName)):
    stuShiftAvailability = []
    stuAvailability = allAvailability[i]
    #loop all shifts
    for j in shifts:
        shiftstartTime = startTime[j]
        shiftLength = shiftslength[j]
        day = int(days[j]-1)
        startTimeSlot = int((shiftstartTime-7)/0.5)
        lengthSlot = int(shiftLength/0.5)
        # get student availability for the specific day
        stuDayAvailability = stuAvailability[day]
        # loop one single shift
        available = 1;
        for k in list(range(startTimeSlot,startTimeSlot+lengthSlot)):
            if(stuDayAvailability[k] ==0):
                available=0
                break
        stuShiftAvailability.append(available)
    a.append(stuShiftAvailability)

notavailable = []
for i in shifts:
    for j in range(i, len(shifts)):
        if days[i] == days[j] and i !=j:
            if startTime[i] <= startTime[j] and endTime[i] > startTime[j]:
                notavailable.append([i,j])
            elif endTime[j] > startTime[i] and endTime[j] <= endTime[i]:
                notavailable.append([i,j])

In [6]:
del allAvailability
# model building
from gurobipy import *
import statistics
m = Model()
# initial variable
#x[i][j] assign student i to shift j
x=[]
for i in range(len(studentName)):
    y = [None]*len(shifts)
    x.append(y)
    for j in shifts:
        x[i][j] = m.addVar(vtype=GRB.BINARY)
m.update()

# add constrain
for i in range(len(studentName)):
    expr = LinExpr()
    for j in shifts:
        if a[i][j] ==0:
            m.addConstr(x[i][j]==0)
        #expr.add(x[i][j],shiftslength[j])
    expr.addTerms(shiftslength,[x[i][j] for j in shifts])
    m.addConstr(expr<=W_max)
    m.addConstr(expr>=W_min)

    for k in notavailable:
        m.addConstr(x[i][k[0]]+x[i][k[1]] <= 1)

for j in shifts:
    expr = LinExpr()
    expr.addTerms([1]*len(studentName),[x[i][j] for i in range(len(studentName))])
    #
    #for i in range(len(studentName)):
    #    expr.add(x[i][j],1)
    m.addConstr(expr<=1)

# goal
# F1
temp = [None]*len(studentName)
F1 = 0
F2 = 0
for i in range(len(studentName)):
    scheduleTime = 0
    stuStation = []
    unique = []
    lateNightshift = 0
    for j in shifts:
        if x[i][j] == 1:
            scheduleTime += x[i][j]*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
            
            if shiftstations[j] == "Late Night":
                lateNightshift+=1
                
    temp[i] = m.addVar()
    m.update()
    m.addConstr(temp[i]>=(scheduleTime-p[i]))
    m.addConstr(temp[i]>=-(scheduleTime-p[i]))
    F1 += temp[i]
    F2 += len(stuStation)-lateNightshift-len(unique)

# Define the objective function
ohm_1 = 3
ohm_2 = 1
#m.Params.MIPGapAbs =2.5
#m.Params.MIPFocus=3
#m.Params.ImproveStartTime = 50
#m.Params.TimeLimit = 1000
m.setObjective((ohm_1*F1+ ohm_2*F2), GRB.MINIMIZE)
m.optimize()

Optimize a model with 225085 rows, 26588 columns and 567497 nonzeros
Variable types: 92 continuous, 26496 integer (26496 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [3e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 220276 rows and 13943 columns
Presolve time: 2.39s
Presolved: 4809 rows, 12645 columns, 70942 nonzeros
Variable types: 0 continuous, 12645 integer (12471 binary)

Root relaxation: objective 2.484575e+04, 1731 iterations, 0.15 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 24845.7500    0    4          - 24845.7500      -     -    3s
H    0     0                    24857.750000 24845.7500  0.05%     -    3s
H    0     0                    24851.750000 24845.7500  0.02%     -    3s
H    0     0                    24847.250000 24845.7500  0.01%     -    3s

Explore

In [27]:
for i in range(0,len(studentName)):
    listVar = x[i]
    result = []
    for var in listVar:
        result.append(var.X)
   # print(result)

In [26]:
workbook = xlsxwriter.Workbook('assignSchedule.xlsx')
sheet = workbook.add_worksheet('schedule')
sheet.set_column('B:B', 20)
sheet.set_column('C:C', 14)
sheet.write(0,0,"Shift")
sheet.write(0,1,"Time")
sheet.write(0,2,"Station")
sheet.freeze_panes(1, 3)
cell_format = workbook.add_format({'bold': True, 'bg_color': '#99ff99'})
for k, value in enumerate(shifts):
    sheet.write(int(k)+1, 0, k+1)
    sheet.write(int(k)+1, 1, dayFormating(days[k])+":"+timeFormating(startTime[k])+"-"+timeFormating(endTime[k]))
    sheet.write(int(k)+1, 2, shiftstations[k])
for j in range(0,len(studentName)):
    sheet.write(0,3+j,studentName[j])
    f = open("schedule/"+studentName[j]+".txt", "w")
    f.write('Student Employee '+studentName[j]+":\n")
    writeMessage = ''
    totalHour = 0
    for i in shifts:
        if (type(x[j][i]) != int) and (x[j][i].x==1) :
            totalHour += x[j][i].x*shiftslength[i]
            sheet.write(i+1,3+j,"Assign",cell_format)
            writeMessage += dayFormating(days[i])+": "+timeFormating(startTime[i])+"-"+\
                            timeFormating(endTime[i])+", ("+str(shiftstations[i])+")\n"
    f.write('Total hour: '+str(totalHour)+'\n')
    f.write(writeMessage)
    f.close()
workbook.close()
#check objective function 1
print("Result for F1")
difference = 0
print("Student: Total time:\tPrefer:\tAbsolute Difference")
for j in range(0,len(studentName)):
    writeMessage = ''
    totalstuHour = 0
    for i in shifts:
        if (type(x[j][i]) != int) and (x[j][i].x==1) :
            totalstuHour += x[j][i].x*shiftslength[i]
    difference+= abs(totalstuHour-p[j])
    print(studentName[j]+"\t "+str(totalstuHour)+"\t\t"+str(p[j])+"\t"+str(abs(totalstuHour-p[j])))
print("Total difference is "+str(difference))
print()
print()
# check objective function 2
print("Result for F2")
totalStation = 0
totalunique = 0
print("Student: No. shifts:\tNo. different station:\tDifference")
for i in range(len(studentName)):
    stuStation = []
    unique = []
    for j in shifts:
        if x[i][j].x == 1:
            scheduleTime += x[i][j].x*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
    totalStation+=len(stuStation)
    totalunique+=len(unique)
    print(studentName[i]+"\t "+str(len(stuStation))+"\t\t"+str(len(unique))+"\t\t\t"+str(len(stuStation)- len(unique)))
print("Total\t "+str(totalStation)+"\t\t"+str(totalunique)+"\t\t\t"+str(totalStation-totalunique))

F1 = 0
F2 = 0
for i in range(len(studentName)):
    scheduleTime = 0
    stuStation = []
    unique = [] 
    for j in shifts:
        if x[i][j].x == 1:
            scheduleTime += x[i][j].x*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
    F1 += abs(scheduleTime-p[i])#*(scheduleTime-p[i])
    F2 += len(stuStation)-len(unique)#len(stuStation)-lateNightshift-len(unique)#len(stuStation)-len(unique)
print()
print('F1: '+str(F1))
print('F2: '+str(F2))
ohm_1 = 3
ohm_2 = 1
print('for each student employee:'+str((ohm_1*F1+ ohm_2*F2)))
studentworkbook = xlsxwriter.Workbook('assignstudentsSchedule.xlsx')
studentsheet = studentworkbook.add_worksheet('schedule')
studentsheet.set_column('B:B', 20)
studentsheet.set_column('C:C', 14)
studentsheet.write(0,0,"Shift")
studentsheet.write(0,1,"Time")
studentsheet.write(0,2,"Station")
studentsheet.write(0,3,"Student")
for k, value in enumerate(shifts):
    studentsheet.write(int(k)+1, 0, k+1)
    studentsheet.write(int(k)+1, 1, dayFormating(days[k])+":"+timeFormating(startTime[k])+"-"+timeFormating(endTime[k]))
    studentsheet.write(int(k)+1, 2, shiftstations[k])
    for i in range(len(studentName)):
        if x[i][k].x == 1:
            studentsheet.write(int(k)+1,3,studentName[i])
    
studentworkbook.close()

Result for F1
Student: Total time:	Prefer:	Absolute Difference
1	 10.0		10.0	0.0
10	 10.5		15.0	4.5
11	 10.25		13.0	2.75
12	 14.75		15.0	0.25
13	 10.25		10.0	0.25
14	 10.5		13.5	3.0
15	 16.0		16.0	0.0
16	 12.0		14.0	2.0
17	 10.5		12.0	1.5
18	 10.25		12.0	1.75
19	 10.5		12.5	2.0
2	 11.5		13.5	2.0
20	 10.0		13.0	3.0
21	 10.75		14.0	3.25
22	 10.0		16.0	6.0
23	 11.75		17.0	5.25
24	 11.0		12.0	1.0
25	 10.25		11.0	0.75
26	 10.0		13.5	3.5
27	 10.5		15.0	4.5
28	 10.0		12.0	2.0
29	 10.5		15.0	4.5
3	 10.5		13.0	2.5
30	 10.75		14.0	3.25
31	 10.0		13.5	3.5
32	 10.25		14.5	4.25
33	 12.0		13.0	1.0
34	 11.25		12.0	0.75
35	 13.5		14.0	0.5
36	 11.75		12.0	0.25
37	 14.5		17.0	2.5
38	 14.0		14.0	0.0
39	 12.5		15.5	3.0
4	 10.0		11.0	1.0
40	 10.25		13.0	2.75
41	 10.0		10.0	0.0
42	 11.0		11.0	0.0
43	 10.0		13.5	3.5
44	 11.25		11.5	0.25
45	 10.5		12.5	2.0
46	 11.0		14.5	3.5
47	 10.0		10.0	0.0
48	 10.0		10.5	0.5
49	 11.25		13.5	2.25
5	 10.75		12.5	1.75
50	 10.0		10.0	0.0
51	 10.5		15.0	4.5
52	 10.5		14.0	3.5


In [23]:
# print remaining shifts
print("Shift\tStart Time\tEnd Time\tLength\tDay\tStation")
n = 0
for j in shifts:
    assign = 0
    for i in range(len(studentName)):
        if x[i][j].x==1:
            assign=1
            break
    if assign == 0:
        n+=1
        print(j,"\t",timeFormating(startTime[j]),"\t",timeFormating(endTime[j]),"\t",shiftslength[j],"\t",dayFormating(days[j]),"\t",shiftstations[j])
print("Total number of unassigned shifts: ",n)

Shift	Start Time	End Time	Length	Day	Station
Total number of unassigned shifts:  0
